# STARR Project - General Data Collection Routine
Author: George Gorospe

Note: This notebook draws heavily from Nvidia's jetbot code base found at: https://github.com/NVIDIA-AI-IOT/jetbot

We can use this basic routine for collecting and labeling different types of data.
Start by entering the name of the object you're photographing. The routine will create a new folder in your data sets directory for the data you collect.

You'll want to take multiple photos of the object all by itself in differient orientations, locations, and lighting conditions. a good combination of all of these will ensure a robust AI capable of identifying the object with high accuracy.

If you are collecting location data, i.e. gym, library, classroom, make sure to take photos of all the objects that will always be in the room. This means taking photos of the walls, desks, tables, trashcans, but not backpacks, coats, or note books.

Final Note: This is experimental software, it has been tested in limited environments and may not do everything we expect. Use it, experiment with it, and feel free to change it. If you break the software you can always download it again from the source and experiment more.


### Display live camera feed

To start, we'll initialize and display the feed from our camera.
Important part here is that we size the image to fit what our neural network.
For certain tasks it may be better to collect larger images then downscale later.

In [ ]:
# First we'll import some tools to help us get the camera started and to make this note book interactive
import traitlets
import ipywidgets.widgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import image, display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Next we want to label the data we'll be collecting:
use the text box to create a label.

In [ ]:
import os  # This library allows us to explore and modify the file structure on the Nano
# Creating a function to add a folder to our directory
def folder_function(label):
    # Using the "try/except" statement here because the makedirs function can throw an error if the directory exists already
    try:
        os.makedirs('data/'+label)
    except:
        print('Directory no created because it already exists')
    print("Creating data folder: data/"+label)            
    
# Creating the interactive text box widget
interact_manual(folder_function, label=widgets.Text(value='structured_string',placeholder = 'structured_string',description='Enter Label:'));

If you refresh the Jupyter file browser on the left, you should now see a new directory with your label.  

Next, let's create and display some buttons that we'll use to save snapshots
for each class label.  We'll also add some text boxes that will display how many images we've collected so far. 

In [1]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

NameError: name 'widgets' is not defined

Right now, these buttons wont do anything.  We have to attach functions to save images for each category to the buttons' ``on_click`` event.  We'll save the value
of the ``Image`` widget (rather than the camera), because it's already in compressed JPEG format!

To make sure we don't repeat any file names (even across different machines!) we'll use the ``uuid`` package in python, which defines the ``uuid1`` method to generate
a unique identifier.  This unique identifier is generated from information like the current time and the machine address.

In [ ]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

Great! Now the buttons above should save images to the ``free`` and ``blocked`` directories.  You can use the Jupyter Lab file browser to view these files!

Now go ahead and collect some data 

1. Place the robot in a scenario where it's blocked and press ``add blocked``
2. Place the robot in a scenario where it's free and press ``add free``
3. Repeat 1, 2

> REMINDER: You can move the widgets to new windows by right clicking the cell and clicking ``Create New View for Output``.  Or, you can just re-display them
> together as we will below

Here are some tips for labeling data

1. Try different orientations
2. Try different lighting
3. Try varied object / collision types; walls, ledges, objects
4. Try different textured floors / objects;  patterned, smooth, glass, etc.

Ultimately, the more data we have of scenarios the robot will encounter in the real world, the better our collision avoidance behavior will be.  It's important
to get *varied* data (as described by the above tips) and not just a lot of data, but you'll probably need at least 100 images of each class (that's not a science, just a helpful tip here).  But don't worry, it goes pretty fast once you get going :)

In [ ]:
display(image)
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

## Next

Once you've collected enough data, we'll need to copy that data to our GPU desktop or cloud machine for training.  First, we can call the following *terminal* command to compress
our dataset folder into a single *zip* file.

> The ! prefix indicates that we want to run the cell as a *shell* (or *terminal*) command.

> The -r flag in the zip command below indicates *recursive* so that we include all nested files, the -q flag indicates *quiet* so that the zip command doesn't print any output

In [ ]:
!zip -r -q dataset.zip dataset

You should see a file named ``dataset.zip`` in the Jupyter Lab file browser.  You should download the zip file using the Jupyter Lab file browser by right clicking and selecting ``Download``.

Next, we'll need to upload this data to our GPU desktop or cloud machine (we refer to this as the *host*) to train the collision avoidance neural network.  We'll assume that you've set up your training
machine as described in the JetBot WiKi.  If you have, you can navigate to ``http://<host_ip_address>:8888`` to open up the Jupyter Lab environment running on the host.  The notebook you'll need to open there is called ``collision_avoidance/train_model.ipynb``.

So head on over to your training machine and follow the instructions there!  Once your model is trained, we'll return to the robot Jupyter Lab enivornment to use the model for a live demo!